In [1]:
from ff_environment import ForceField
import numpy as np

In [2]:
env = ForceField()
env_info = env.reset()

In [ ]:
# size of each action
action_size = env.action_
print('Size of each action:', action_size)



# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])